In [22]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
import re
import time
import math
from alive_progress import alive_bar

# NYT API key
keyNYT = "BwemetlgrQFLmyML7BqSZqCjYgMvyANE"


In [19]:
# Directories
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))
PARENT_DIR = os.path.dirname(ROOT_DIR)

# Query setup function
def NYT(page):
    return requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?&api-key=" + keyNYT +
  "&begin_date=20220201" +
  "&sort=newest" +
  "&fl=web_url%2Cheadline%2Cword_count%2Csnippet%2Clead_paragraph%2Cpub_date%2Ctype_of_material%2Cdocument_type" + 
  "&fq=type_of_material%3A(%22News%22)&page=" + str(page) +
  "&q=ukraine")

def numArticlesInPage(json):
    if json["response"]["meta"]["hits"] - json["response"]["meta"]["offset"] >= 10:
        return 10
    else:
        return json["response"]["meta"]["hits"] - json["response"]["meta"]["offset"] + 1

# Instancing a query to fetch basic information
json_NYT = NYT(1).json()
#numPages = json_guardian["response"]["headerspages"]
numArticles = json_NYT["response"]["meta"]["hits"]
numPages = ceil(numArticles/10)
#print(json.dumps(json_NYT, indent=2))

print(f"-> There are {numArticles} articles in {numPages} pages for the period and tags specified.")

-> There are 3065 articles in 307 pages for the period and tags specified.


In [18]:
print(json.dumps(json_NYT, indent=2))

{
  "status": "OK",
  "copyright": "Copyright (c) 2022 The New York Times Company. All Rights Reserved.",
  "response": {
    "docs": [
      {
        "web_url": "https://www.nytimes.com/2022/06/20/world/asia/ukraine-war-climate-crisis-activists-protests.html",
        "snippet": "Led by young women from Eastern Europe, they are cornering Europe\u2019s leaders and pressing them for a total energy embargo on Russia \u2014 to end the fighting and to save the planet.",
        "lead_paragraph": "BRUSSELS \u2014 Emmanuel Macron, the president of France, had just finished a speech at a major conference on Europe.",
        "headline": {
          "main": "New Brand of Activist Takes Aim at Ukraine War and Climate Crisis, Together",
          "kicker": null,
          "content_kicker": null,
          "print_headline": "New Brand of Activist Links 2 Causes: War and Climate Change",
          "name": null,
          "seo": null,
          "sub": null
        },
        "pub_date": "2022-06-2

In [23]:

urls = []
titles = []
bodies = []
dates = []
snippets = []


# Scraper
with alive_bar(numArticles, title="-> API Query", spinner="dots_waves", bar="smooth", force_tty=True) as bar:

    # Going through all pages available for the query
    for i in range(1,numPages):

        time.sleep(6) #NYT request limit
        json_NYT = NYT(i).json()

        # Going through all articles in a page
        for j in range(0,numArticlesInPage(json_NYT)):
            
            urls.append(json_NYT["response"]["docs"][j]["web_url"])
            dates.append(json_NYT["response"]["docs"][j]["pub_date"])

            titles.append(json_NYT["response"]["docs"][j]["headline"]["main"])
            #titles.append(re.sub(r"\|.*$", "", title)) # removing authors from titles

            body = BeautifulSoup(json_NYT["response"]["docs"][j]["lead_paragraph"], "html.parser").get_text()
            #body = replace_all(body, rep)  # replacing substrings
            bodies.append(re.sub(r"[\t\r\n]", "", body)) # removing line breaks

            snippet = BeautifulSoup(json_NYT["response"]["docs"][j]["snippet"], "html.parser").get_text() 
            snippets.append(re.sub(r"^[^—]+—\s*", "", snippet)) 
            bar()

# Transforming fetched info to dataframe
data = pd.DataFrame({"URL": urls, "Date": dates, "Title": titles, "Text": bodies, "Snippet" : snippets})

# Saving to csv
os.makedirs(PARENT_DIR + "/data", exist_ok=True)
data.to_csv(PARENT_DIR + "/data/NYT.csv", index=True, header=True)

print(f"-> {len(data)} articles fetched successfully!")


on 412: /home/r2/code/ukraine-media/.venv/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
          warnings.warn(


-> API Query |████████████▍                           | ⠠⢀⡀⠄⠂ 950/3065 [31%] in 13:34 (1.2/s, eta: 30:15) 

In [ ]:
data.head(25)